In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import geopandas as gpd
import geoplot.crs as gcrs
import geoplot as gplt
import re
from geopy.distance import great_circle
from shapely.geometry import Polygon, LineString, Point, MultiPolygon
from shapely import wkt
import folium

In [2]:
# Load the cleaned datasets for analysis and visualization
df = pd.read_csv('data/df_combined_cleaned_20250902.csv')
df2 = pd.read_csv('data/df_combined2_cleaned_20250902.csv')

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_1756/947270512.py:2: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/df_combined_cleaned_20250902.csv')


In [3]:
df['source'].value_counts()

source
Mapillary    1803853
311           328980
311            31155
Name: count, dtype: int64

In [4]:
df2['source'].value_counts()

source
Mapillary    378008
311          325788
Name: count, dtype: int64

In [5]:
# Convert data type of 'source' column to string
df['source'] = df['source'].astype(str)
df[df['source'] == '311'].shape

(360135, 12)

In [6]:
df['source'].value_counts()

source
Mapillary    1803853
311           360135
Name: count, dtype: int64

In [7]:
df.head()

,timestamp,bboxid,latitude,longitude,ground_truth,category,source,geometry_bbox,center_latlon,year,month,day
0,2016-11-09 14:15:57.000,bbox_10_14,37.720555,-122.484861,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
1,2016-11-09 14:15:58.000,bbox_10_14,37.720457,-122.484861,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
2,2016-11-09 14:15:59.000,bbox_10_14,37.720360,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
3,2016-11-09 14:16:00.000,bbox_10_14,37.720258,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
4,2016-11-09 14:16:01.000,bbox_10_14,37.720156,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9


In [8]:
# Convert df to geo data frame
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=4326)
df2 = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2.longitude, df2.latitude), crs=4326)

In [9]:
print(df.shape)
print(df2.shape)

(2163988, 13)
(703796, 13)


In [ ]:
from tqdm import tqdm

# Create a 311 map centered at San Francisco
m2 = folium.Map(location=[37.7749, -122.4194], zoom_start=12, tiles='CartoDB positron')

# Add points to the map
for _, row in tqdm(df2[(df2['source']=='311') & (df2['ground_truth']==1)].iterrows(), 
total=df2[(df2['source']=='311') & (df2['ground_truth']==1)].shape[0]):
    folium.CircleMarker(
        location = [row['latitude'], row['longitude']],
        radius = .3,
        color = "#e14b31",
        fill = True,
        opacity = 0.3,
    ).add_to(m2)

for _, row in tqdm(df2[(df2['source']=='311') & (df2['ground_truth']==0)].iterrows(), 
total=df2[(df2['source']=='311') & (df2['ground_truth']==0)].shape[0]):
    folium.CircleMarker(
        location = [row['latitude'], row['longitude']],
        radius = .3,
        color = "#1984c5",
        fill = True,
        opacity = 0.3,
    ).add_to(m2)

100%|██████████| 23264/23264 [00:02<00:00, 9326.05it/s] 


In [22]:
from tqdm import tqdm

# Create a 311 map centered at San Francisco
m3 = folium.Map(location=[37.7749, -122.4194], zoom_start=12, tiles='CartoDB positron')

# Add points to the map
for _, row in tqdm(df2[(df2['source']=='Mapillary') & (df2['ground_truth']==0)].iterrows(), 
total=df2[(df2['source']=='Mapillary') & (df2['ground_truth']==0)].shape[0]):
    folium.CircleMarker(
        location = [row['latitude'], row['longitude']],
        radius = .3,
        color = "#1984c5",
        fill = True,
        opacity = 0.3,
    ).add_to(m3)

for _, row in tqdm(df2[(df2['source']=='Mapillary') & (df2['ground_truth']==1)].iterrows(), 
total=df2[(df2['source']=='Mapillary') & (df2['ground_truth']==1)].shape[0]):
    folium.CircleMarker(
        location = [row['latitude'], row['longitude']],
        radius = .3,
        color = "#e14b31",
        fill = True,
        opacity = 0.8,
    ).add_to(m3)


100%|██████████| 208/208 [00:00<00:00, 1467.63it/s]


In [23]:
# Save the maps to HTML files
m2.save('311_ground_truth_map_20250906.html')
m3.save('mapillary_ground_truth_map_20250906.html')

In [42]:
# Load the amenities data for San Francisco from OpenStreetMap
df_map = pd.read_csv('data/sf_osm_amenities_2016_2024.csv')

# Filter the dataframe to only include data from the snapshot date '2024-05-31'
df_map = df_map[df_map['snapshot_date']=='2024-05-31']

In [43]:
# Filter the dataframe to only include amenities of interest
amenities_of_interest = ['bench', 'restaurant', 'fast_food', 'place_of_worship', 'school', 'bank', 
                         'atm', 'social_facility', 'shelter', 'university', 'college', 'police', 'cinema']

df_map = df_map[df_map['amenity'].isin(amenities_of_interest)]
df_map['amenity'].value_counts()

amenity
bench               3060
restaurant          1831
fast_food            417
place_of_worship     400
school               271
bank                 230
atm                  171
social_facility       73
shelter               46
university            31
college               19
police                18
cinema                17
Name: count, dtype: int64

In [ ]:
from tqdm import tqdm

# Create a amenity map centered at San Francisco
m4 = folium.Map(location=[37.7749, -122.4194], zoom_start=12, tiles='CartoDB positron')

# Add amenity points to the map with different colors based on amenity type
# Iterate through each row in the dataframe and add a CircleMarker for each amenity type
for _, row in tqdm(df_map.iterrows(), total=df_map.shape[0]):
    color = ""  # Default color
    if row['amenity'] == 'restaurant':
        color = "#1984c5"
    elif row['amenity'] == 'bench':
        color = "#e14b31"
    elif row['amenity'] == 'fast_food':
        color = "#2ca02c"
    elif row['amenity'] == 'place_of_worship':
        color = "#ff7f0e"
    elif row['amenity'] == 'school':
        color = "#9467bd"
    elif row['amenity'] == 'bank':
        color = "#8c564b"
    elif row['amenity'] == 'atm':
        color = "#d62728"
    elif row['amenity'] == 'social_facility':
        color = "#17becf"
    elif row['amenity'] == 'shelter':
        color = "#bcbd22"
    elif row['amenity'] == 'university':
        color = "#ffbb78"
    elif row['amenity'] == 'college':
        color = "#aec7e8"
    elif row['amenity'] == 'police':
        color = "#c49c94"
    elif row['amenity'] == 'cinema':
        color = "#98df8a"
    folium.CircleMarker(
        location = [row['lat'], row['lon']],
        radius = 1.5,
        color = color,
        fill = True,
        opacity = 0.5,
    ).add_to(m4)

# Add a legend to the map for the different amenity types
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 170px; height: 300px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; padding: 10px;">
     <b>Amenity Types</b><br>
     <i style="background:#1984c5; width:10px; height:10px; display:inline-block;"></i> Restaurant<br>
     <i style="background:#e14b31; width:10px; height:10px; display:inline-block;"></i> Bench<br>
     <i style="background:#2ca02c; width:10px; height:10px; display:inline-block;"></i> Fast Food<br>
     <i style="background:#ff7f0e; width:10px; height:10px; display:inline-block;"></i> Place of Worship<br>
     <i style="background:#9467bd; width:10px; height:10px; display:inline-block;"></i> School<br>
     <i style="background:#8c564b; width:10px; height:10px; display:inline-block;"></i> Bank<br>
     <i style="background:#d62728; width:10px; height:10px; display:inline-block;"></i> ATM<br>
     <i style="background:#17becf; width:10px; height:10px; display:inline-block;"></i> Social Facility<br>
     <i style="background:#bcbd22; width:10px; height:10px; display:inline-block;"></i> Shelter<br>
     <i style="background:#ffbb78; width:10px; height:10px; display:inline-block;"></i> University<br>
     <i style="background:#aec7e8; width:10px; height:10px; display:inline-block;"></i> College<br>
     <i style="background:#c49c94; width:10px; height:10px; display:inline-block;"></i> Police<br>
     <i style="background:#98df8a; width:10px; height:10px; display:inline-block;"></i> Cinema<br>
</div>
'''

# Add the legend HTML to the map with the updated title "Amenity Types"
m4.get_root().html.add_child(folium.Element(legend_html))

# Save the amenity map to an HTML file
m4.save('amenity_map_20250906.html')


100%|██████████| 6584/6584 [00:00<00:00, 10225.02it/s]


In [53]:
# Load the amenities data for San Francisco from OpenStreetMap
df_map = pd.read_csv('data/sf_osm_amenities_2016_2024.csv')

# Filter the dataframe to only include data from the snapshot date '2024-05-31'
df_map = df_map[df_map['snapshot_date']=='2024-05-31']

In [54]:
df_map.head()

,snapshot_date,osm_type,osm_id,name,feature_type,amenity,social_facility,highway,bridge,tags,lat,lon
39915,2024-05-31,node,140982581.0,Opera Plaza Cinema,amenity:cinema,cinema,NaN,NaN,NaN,"{'wikidata': 'Q44634616', 'addr:city': 'San Fr...",37.781376,-122.421561
39916,2024-05-31,node,281652642.0,John O'Connell High School,amenity:school,school,NaN,NaN,NaN,"{'amenity': 'school', 'name': ""John O'Connell ...",37.759543,-122.414183
39917,2024-05-31,node,286132370.0,Mel's Drive-in,amenity:restaurant,restaurant,NaN,NaN,NaN,"{'wheelchair': 'yes', 'name': ""Mel's Drive-in""...",37.799551,-122.437048
39918,2024-05-31,node,286898816.0,Saint Anthony - Immaculate Conception,amenity:school,school,NaN,NaN,NaN,"{'amenity': 'school', 'name': 'Saint Anthony -...",37.747379,-122.413844
39919,2024-05-31,node,302448629.0,Mezze and Mooore,amenity:restaurant,restaurant,NaN,NaN,NaN,"{'addr:city': 'San Francisco', 'addr:country':...",37.768092,-122.424586


In [56]:
# Filter the dataframe to only include "bridge" and "highway_link"
df_map = df_map[df_map['feature_type'].isin(['bridge', 'highway_link'])]

In [59]:
df_map['feature_type'].value_counts()

feature_type
bridge          680
highway_link    511
Name: count, dtype: int64

In [64]:
from tqdm import tqdm

# Create a amenity map centered at San Francisco
m5 = folium.Map(location=[37.7749, -122.4194], zoom_start=12, tiles='CartoDB positron')

# Add structure (bridge and highway_link) points to the map with different colors and shapes based on feature type
for _, row in tqdm(df_map.iterrows(), total=df_map.shape[0]):
    color = ""  # Default color
    if row['feature_type'] == 'bridge':
        color = "#1984c5"
        icon = "bridge"
    elif row['feature_type'] == 'highway_link':
        color = "#e14b31"
        icon = "road"
    folium.Marker(
        location=[row['lat'], row['lon']],
        icon=folium.Icon(color=color, icon=icon, size='small'),
        opacity = 0.5,
    ).add_to(m5)
    # folium.CircleMarker(
    #     location = [row['lat'], row['lon']],
    #     radius = 1.5,
    #     color = color,
    #     fill = True,
    #     opacity = 0.5,
    # ).add_to(m5)

# Add a legend to the map for the different feature types
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 170px; height: 100px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; padding: 10px;">
     <b>Feature Types</b><br>
     <i style="background:#1984c5; width:10px; height:10px; display:inline-block;"></i> Bridge<br>
     <i style="background:#e14b31; width:10px; height:10px; display:inline-block;"></i> Highway Link<br>
</div>
'''

# Add the legend HTML to the map with the updated title "Feature Types"
m5.get_root().html.add_child(folium.Element(legend_html))

# Save the feature map to an HTML file
m5.save('feature_map_20250906.html')


  0%|          | 0/1191 [00:00<?, ?it/s]/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_1756/2882784046.py:17: UserWarning: color argument of Icon should be one of: {'blue', 'beige', 'purple', 'darkgreen', 'darkpurple', 'white', 'darkblue', 'lightblue', 'lightgray', 'cadetblue', 'lightred', 'red', 'green', 'pink', 'black', 'orange', 'gray', 'darkred', 'lightgreen'}.
  icon=folium.Icon(color=color, icon=icon, size='small'),
100%|██████████| 1191/1191 [00:00<00:00, 8754.52it/s]
